In [122]:
import json
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [1]:
%cd F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\data_553\project\REJ_data\REJ_data

F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\data_553\project\REJ_data\REJ_data


### Load Training and Testing data

In [120]:
import os
os.chdir('C:/Users/l/ReviewClassifier4J/ReviewClassifier4J/data')

In [11]:
# from all.json
with open('all.json', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    #alljson=json_data[0:400]
df=pd.DataFrame.from_dict(json_normalize(json_data), orient='columns')

In [13]:
%cd F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\data_553\project\ReviewClassifier4J\ReviewClassifier4J\data

F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\data_553\project\ReviewClassifier4J\ReviewClassifier4J\data


In [131]:
# Bug data
with open('Bug_tt.json', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    #alljson=json_data[0:400]
df_bug=pd.DataFrame.from_dict(json_normalize(json_data), orient='columns')

In [132]:
# Bug report data
with open('Feature_tt.json', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    #alljson=json_data[0:400]
df_feature=pd.DataFrame.from_dict(json_normalize(json_data), orient='columns')

In [133]:
# Feature data
with open('Rating_tt.json', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    #alljson=json_data[0:400]
df_rating=pd.DataFrame.from_dict(json_normalize(json_data), orient='columns')

In [134]:
# Bug report data
with open('UserExperience_tt.json', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    #alljson=json_data[0:400]
df_user_experience = pd.DataFrame.from_dict(json_normalize(json_data), orient='columns')

# Reorganize Testing Data

In [ ]:
import os
os.chdir('C:/Users/l/ReviewClassifier4J/ReviewClassifier4J')

In [78]:
# Testing data
import pandas as pd
p=pd.read_csv("data/dataset3_with_nlp_techniques.csv")

In [79]:
labelled=pd.read_csv("data/labelled.csv")
p["label"]= labelled["label"]
p.columns.values[0] = "index"

In [80]:
def append_str_in_list(s): return ' '.join([str(elem) for elem in s])
new_comment_list = []
for row in p.itertuples():
    each_text = getattr(row, "stopword")
    index = int(getattr(row, "index"))
    # get all non-english reviews rows
    final_value = append_str_in_list(ast.literal_eval(each_text) )
    new_comment_list.append(final_value)
p['stopwords_removal'] = new_comment_list

In [81]:
def append_str_in_list(s): return ' '.join([str(elem) for elem in s])
new_comment_list = []
for row in p.itertuples():
    each_text = getattr(row, "lemmatizer")
    index = int(getattr(row, "index"))
    # get all non-english reviews rows
    final_value = append_str_in_list(ast.literal_eval(each_text) )
    new_comment_list.append(final_value)
p['lemmatized_comment'] = new_comment_list

In [82]:
# stop words and lemmatize
import nltk
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
lemmatizer= WordNetLemmatizer()
def word_lemmatizer(text):
    word_tokens = word_tokenize(text)
    lem_text=[str(lemmatizer.lemmatize(i)) for i in word_tokens]
    return lem_text
p['stopwords_removal_lemmatization']=p['stopwords_removal'].apply(lambda x: word_lemmatizer(x))

In [83]:
def append_str_in_list(s): return ' '.join([str(elem) for elem in s])
new_comment_list = []
for row in p.itertuples():
    each_text = getattr(row, "stopwords_removal_lemmatization")
    index = int(getattr(row, "index"))
    # get all non-english reviews rows
    final_value = append_str_in_list((each_text) )
    new_comment_list.append(final_value)
p['stopwords_removal_lemmatization'] = new_comment_list

In [84]:
df = pd.DataFrame()
df["comment"] = p["processed_text"]
df["rating"] = p["score"]
df["past"] = p["past"]
df["stopwords_removal"] = p["stopwords_removal"]
df["reviewer"] = None
df["id"] = None
df["stemmed"] = p["stemmer"]
df["fee"] = None
df["title"] = None
df["label"]=p["label"]
df["future"] = p["future"]
df["lemmatized_comment"] = p["lemmatized_comment"]
df["sentiScore"] = None
df["sentiScore_neg"] = None
df["reviewId"] = None
df["stopwords_removal_nltk"] = p["stopwords_removal"]
df["present_simple"] = p["present_simple"]
df["dataSource"] = None
df["appId"] = None
df["date"] = p["date_ex"]
df["sentiScore_pos"] = None
df["present_con"] = p["present_con"]
df["length_words"] = p["wordcount"]
df["stopwords_removal_lemmatization"] = p["stopwords_removal_lemmatization"]

In [88]:
%cd F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\553project

F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\553project


In [93]:
df.to_json(r'./test_all_new.json',orient='records')

# Split testing data to 4 binary category files

In [116]:
df1 = df.copy()
new_bug_list =[]
for row in df1.itertuples():
    bug = getattr(row, "label")
    if(bug != "Bug"):
        final_value = "Not_Bug"
    else:
        final_value = "Bug"
    new_bug_list.append(final_value)
df1['label'] = new_bug_list
df1.to_json(r'./Bug.json',orient='records')

In [117]:
df2 = df.copy()
new_feature_list =[]
for row in df2.itertuples():
    bug = getattr(row, "label")
    if(bug != "Feature"):
        final_value = "Not_Feature"
    else:
        final_value = "Feature"
    new_feature_list.append(final_value)
df2['label'] = new_feature_list
df2.to_json(r'./Feature.json',orient='records')

In [118]:
df3 = df.copy()
new_Rating_list =[]
for row in df3.itertuples():
    bug = getattr(row, "label")
    if(bug != "Rating"):
        final_value = "Not_Rating"
    else:
        final_value = "Rating"
    new_Rating_list.append(final_value)
df3['label'] = new_Rating_list
df3.to_json(r'./Rating.json',orient='records')

In [119]:
df4 = df.copy()
new_UserExperience_list =[]
for row in df4.itertuples():
    bug = getattr(row, "label")
    if(bug != "UserExperience"):
        final_value = "Not_UserExperience"
    else:
        final_value = "UserExperience"
    new_UserExperience_list.append(final_value)
df4['label'] = new_UserExperience_list
df4.to_json(r'./UserExperience.json',orient='records')

### Merge Testing To Origninal Data

In [129]:
%cd F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\553project\data

F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\553project\data


In [130]:
df_new_bug = pd.concat([df_bug, df1], sort=False)
df_new_feature = pd.concat([df_feature, df2], sort=False)
df_new_rating = pd.concat([df_rating, df3], sort=False)
df_new_user_experience = pd.concat([df_user_experience, df4], sort=False)

df_new_bug.to_json(r'./Bug_tt.json',orient='records')
df_new_feature.to_json(r'./Feature_tt.json',orient='records')
df_new_rating.to_json(r'./Rating_tt.json',orient='records')
df_new_user_experience.to_json(r'./UserExperience_tt.json',orient='records')